In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").getOrCreate()

In [2]:
flightData2015 = spark\
    .read\
    .option("interSchema", "true")\
    .option("header", "true")\
    .csv("/home/jovyan/data/flight-data/csv/2015-summary.csv")

In [3]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [4]:
sqlWay = spark.sql("""
    SELECT DEST_COUNTRY_NAME, count(1)
    FROM flight_data_2015
    GROUP BY DEST_COUNTRY_NAME
""")

In [7]:
sqlWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#16], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#16, 200), ENSURE_REQUIREMENTS, [id=#33]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#16], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#16] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [6]:
dataFrameWay = flightData2015\
    .groupBy("DEST_COUNTRY_NAME")\
    .count()

In [8]:
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#16], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#16, 200), ENSURE_REQUIREMENTS, [id=#46]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#16], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#16] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


